In [1]:
# stamp_viewer env
import numpy as np
import matplotlib.pyplot as plt
import setigen as stg
from astropy import units as u
# import multiprocessing
# from datetime import datetime

In [2]:
n_ints = 64

n_fine = 2**17 # fine channels - 2^17 for VLASS, 2^19 for high freq
n_pols = 2 # polarizations
n_complex = 2 # parts of complex result
drifts = np.arange(-50, 50.25, .25)

PFB_filepath='/home/nstiegle/raw_data_work/VLASS_response.f32'
f_off=7.63e-06
t_samp=8.338608 / 64
freq=3000

drifts = np.arange(-50, 50.25, .25)

In [3]:
def gen_obs_data(n_fine, n_ints, n_pols, n_complex, PFB_filepath=""):
    """
    gen_obs_data
    Generate one observational coarse channel

    n_fine: Number of fine channels per coarse channel - usually a power of 2
    n_ints: Number of integrations performed, amount of data per frequency bin
    n_pols: Number of polarizations observed - usually 2
    n_complex: Number of complex components to an observation - usually 2 for IQ
    PFB_filepath: Filepath to the PFB response for the observation, or "" if no
                  PFB response should be added

    Returns a numpy array which is (n_ints, n_fine), where each element represents
    the data observed at that frequency and integration, a Chi-Square(4)
    distribution (sum of squares of 4 normal samples)
    """
    # generate fake data, one for each polarization / complex num, then square and sum
    mean = 0
    std_dev = 1
    n_nums_per_pixel = n_pols * n_complex
    data = np.random.normal(loc=mean, scale=std_dev, size=(n_ints, n_fine, n_nums_per_pixel))
    data = np.square(data)
    data =  np.sum(data, axis=2)

    # Add PFB correction
    if PFB_filepath != "": # '/home/nstiegle/raw_data_work/COSMIC_response.f32' for high freq data
        # Read in PFB shape data
        with open(PFB_filepath, 'rb') as f:
            # Use numpy's fromfile method to read in the 32-bit floats
            pfb = np.fromfile(f, dtype=np.float32)

        # Check shapes and apply PFB correction to data
        assert(len(pfb) == data.shape[1])
        before_shape = data.shape
        data *= pfb
        assert(data.shape == before_shape)

    return data

In [4]:
data = gen_obs_data(n_fine, n_ints, n_pols, n_complex, PFB_filepath=PFB_filepath)

In [5]:
data.shape

(64, 131072)

In [16]:
def above_snr_at_drift(data, drift_rate, snr_thresh, f_off, t_samp, freq):
    """
    above_snr_at_drift
    Figure out how many drift lines yield an integrated intensity above the
    given SNR threshold at the given drift rate

    data_shape: Shape of the shared_data object stored in global memory, usually
                (n_ints, n_fine)
    drift_rate: Drift rate to dedrift and integrate at, in Hz/s
    snr_thresh: Threshold in SNR to define a detection
    f_off:      Frequency offset/spacing between fine channels in MHz
    t_samp:     Length in seconds of each integration
    freq:       Frequency of the first fine channel in MHz

    The integer number of drift lines with a positive detection
    """
    
    # Get snr of all signals
    def give_snr(integrated):
        median = np.percentile(integrated, 50)
        top_5 = np.percentile(integrated, 95)
        bottom_5 = np.percentile(integrated, 5)
        center_data = integrated[(integrated < top_5) * (integrated > bottom_5)]
        std_dev = center_data.std()

        return (integrated - median) / std_dev

    # Dedrift
    frame = stg.Frame.from_data(f_off * u.MHz, t_samp * u.s, freq * u.MHz, True, data)
    dedrifted = stg.dedrift(frame, drift_rate)
    integrated = stg.integrate(dedrifted, mode='sum')

    # See how many signals above threshold
    snrs = give_snr(integrated)
    print("Mean of the SNRS:", np.mean(snrs))
    print("Max SNR", np.max(snrs))
    return sum(snrs > snr_thresh)

In [18]:
drift_rate = 0
snr_thresh = 5
result = above_snr_at_drift(data, drift_rate, snr_thresh, f_off, t_samp, freq)
print(f"Number above {snr_thresh} SNR:", result)

Mean of the SNRS: -0.18523299704441532
Max SNR 3.2083276792106914
Number above 5 SNR: 0
